# Линейная регрессия в Keras

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Данные

In [ ]:
df = pd.read_csv('d:\Projects\DeepLearning\DATA\kc_house_data.csv')
df.head()

Описание данных:
* **id** - уникальный идентификатор для каждого продаваемого дома;
* **date** - дата продажи дома;
* **price** - цена продажи дома;
* **bedrooms** - количество спален;
* **bathrooms** - количество ванных комнат;
* **sqft_living** - количество квадратных футов жилой площади;
* **sqft_lot** - количество квадратных футов общей площади;
* **floors** - количество этажей;
* **waterfront** - переменная, которая показывает степень вида на набережную;
* **view** - оценка вида из окон от 0 до 4;
* **condition** - индекс от 1 до 5 оценки состояния квартиры;
* **grade** - оценка от 1 до 13 строительства и дизайна;
* **sqft_above** - квадратные футы жилой площади над уровнем земли;
* **sqft_basement** - квадратные футы жилой площади под уровнем земли;
* **yr_built** - год постройки дома;
* **yr_renovated** - год последней реновации дома;
* **zipcode** - почтовый индекс дома;
* **lat** - широта;
* **long** - долгота;
* **sqft_living15** - средний размер жилой площади ближайших 15 соседей;
* **sqft_lot15** - средняя общая площадь ближайших 15 соседей. 

## Предварительный анализ данных

Пустые значения

In [ ]:
df.isnull().sum()

Пустых значений нет

Статистические характеристики данных

In [ ]:
df.describe().transpose()

График распределения цен на дома

In [ ]:
plt.figure(figsize=(12, 8))
sns.distplot(df['price'])

График распределения количества спален

In [ ]:
sns.countplot(df['bedrooms'])

График зависимости цены от жилой площади

In [ ]:
plt.figure(figsize=(12, 8))
sns.scatterplot(x='price', y='sqft_living', data=df)

График зависимости цены от количества спален

In [ ]:
sns.boxplot(x='bedrooms', y='price', data=df)

### Географические показатели

In [ ]:
plt.figure(figsize=(12, 8))
sns.scatterplot(x='price', y='long', data=df)

In [ ]:
plt.figure(figsize=(12, 8))
sns.scatterplot(x='price', y='lat', data=df)

График зависимости расположения дома и его цены

In [ ]:
plt.figure(figsize=(12, 8))
sns.scatterplot(x='long', y='lat', data=df, hue='price')

Удалим выбросы

In [ ]:
df.sort_values('price', ascending=False).head(20)

Удалим 1% самых дорогих домов

In [ ]:
len(df) * 0.01

In [ ]:
non_top_1_perc = df.sort_values('price', ascending=False).iloc[216:]

Нарисуем график на новых данных

In [ ]:
plt.figure(figsize=(12, 8))
sns.scatterplot(x='long', y='lat', data=non_top_1_perc, hue='price', palette='RdYlGn', edgecolor=None, alpha=0.2)

### Прочие признаки

In [ ]:
sns.boxplot(x='waterfront', y='price', data=df)

## Исследование признаков

In [ ]:
df.head()

 
Удалим колонку id, она не предсталяет информационной ценности

In [ ]:
df = df.drop('id', axis=1)

Изучим колонку date. Поскольку она сейчас представляет из себя строку - преобразуем ее в тип datetime

In [ ]:
df['date'] = pd.to_datetime(df['date'])

In [ ]:
df['date']

Теперь можно извлечь информацию о годе и месяце даты

In [ ]:
df['year'] = df['date'].apply(lambda date: date.year)
df['month'] = df['date'].apply(lambda date: date.month)

In [ ]:
df.head()

Построим график зависимости цены от месяца продажи

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(x='month', y='price', data=df)

In [ ]:
df.groupby('month').mean()['price'].plot()

Построим такой же график зависимости цены от года

In [ ]:
df.groupby('year').mean()['price'].plot()

Теперь можно просто удалить колонку даты

In [ ]:
df = df.drop('date', axis=1)

In [ ]:
df.columns

In [ ]:
df.head()

Изучим колонку zipcode. Фактически она является категориальной, но если мы поставим ее в модель в текущем виде, то модель воспримет ее как числовую.

In [ ]:
df['zipcode'].value_counts()

Итак, у нас есть 70 уникальных zipcode

Но сейчас мы просто удалим эту колонку

In [ ]:
df = df.drop('zipcode', axis=1)

Изучим колонку "год реновации дома"

In [ ]:
df['yr_renovated'].value_counts()

0 фактически не является годом реновации, но смысл 0 хорош тем, что представляется как минимальный год реновации, что аналогичной ее отсуствию.

Рассмотрим колонку sqft_basement

In [ ]:
df['sqft_basement'].value_counts()

## Разделение данных

In [ ]:
X = df.drop('price', axis=1).values
y = df['price'].values

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

## Масштабирование

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler = MinMaxScaler()

In [ ]:
X_train = scaler.fit_transform(X_train)

In [ ]:
X_test = scaler.transform(X_test)

## Создание модели

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam

In [ ]:
model = Sequential()

In [ ]:
X_train.shape

In [ ]:
model.add(Dense(19, activation='relu'))
model.add(Dense(19, activation='relu'))
model.add(Dense(19, activation='relu'))
model.add(Dense(19, activation='relu'))

model.add(Dense(1))

In [ ]:
model.compile(optimizer='adam', loss='mse')

## Обучение модели

In [ ]:
model.fit(x=X_train, y=y_train, 
          validation_data=(X_test, y_test),
          batch_size=128,
          epochs=400)

Построим график потерь модели

In [ ]:
losses = pd.DataFrame(model.history.history)

In [ ]:
losses.plot()

## Оценка модели

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score

In [ ]:
predictions = model.predict(X_test)

In [ ]:
mean_squared_error(y_test, predictions)

In [ ]:
np.sqrt(mean_squared_error(y_test, predictions))

In [ ]:
mean_absolute_error(y_test, predictions)

In [ ]:
df['price'].mean()

In [ ]:
explained_variance_score(y_test, predictions)

In [ ]:
plt.scatter(y_test, predictions)

plt.plot(y_test, y_test, 'r')

In [ ]:
errors = y_test.reshape(6480, 1) - predictions

In [ ]:
sns.distplot(errors)

## Предсказания модели

In [ ]:
single_house = df.drop('price', axis=1).iloc[0]

In [ ]:
single_house = scaler.transform(single_house.values.reshape(-1, 19))

In [ ]:
single_house

In [ ]:
model.predict(single_house)

In [ ]:
df.iloc[0]